In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
from pandas import read_csv
import datetime
import warnings
warnings.filterwarnings("ignore")

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD,RMSprop, Adam,adadelta
from keras.utils import np_utils
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier

# Use scikit-learn to grid search 
import numpy
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

np.random.seed(42) 

In [ ]:
### Get the data 
#training
train = read_csv("./features_2iter/train_v3.csv")
array_train = train.values
X= array_train[:,3:200]
y = array_train[:,0]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
print(X_train.shape[0], 'train samples')
print(pd.value_counts(pd.Series(y_train)))

print(X_test.shape[0], 'test samples')
print(pd.value_counts(pd.Series(y_test)))


# Generate dummy target 

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train_dummy = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
y_test_dummy = np_utils.to_categorical(encoded_Y)


### Get the submission data
test = read_csv("./features_2iter/test_v3.csv")
array_test = test.values
X_sub= array_test[:,2:200]
print(X_sub.shape[0], 'Submission samples')



In [ ]:
#Set parameters
NB_CLASSES=3
N_HIDDEN= X_train.shape[1]
DROPOUT=0.5
#OPTIMIZER=SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
#OPTIMIZER=Adam()


# Function to create model, required for KerasClassifier
def create_model(neurons=50,activation='relu',  dropout_rate=0.0, optimizer='rmsprop', init_mode='glorot_uniform'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(N_HIDDEN,) ,kernel_initializer=init_mode,activation=activation))
    #model.add(Activation(activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    #model.add(Activation(activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(NB_CLASSES))
    model.add(Activation('softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam','adadelta']
init_mode = ['glorot_uniform', 'normal', 'uniform']
activation = ['softplus', 'relu', 'tanh', 'linear']
dropout_rate = [0.0, 0.5, 0.8]
neurons=[50,100,200]
epochs = [10,20,30]
batches = [10, 50,100]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, 
                  init_mode=init_mode, activation=activation,dropout_rate=dropout_rate,neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train_dummy)



In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [25]:
help(GridSearchCV)


Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either estimator needs to provide a ``score`` function,
 |      or ``scoring`` must be passed.
 |  
 |  param_grid : dict or list of dictionaries
 |      Dictionary with parameters names (string) as ke